# 1. Import Dependencies

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

## How to get new data
Remember to do a backup before to proceed, look at backup\appunti.txt.

To generate new data pen scripts\getDataHandGesture.py and edit labels adding new gestures. 

Execute it to export data in src\dataHandGesture

## How to export and use the generated model
At the end of the execution the generated model will be here: Tensorflow\workspace\models

Then, copy the name of the folder inside Tensorflow\workspace\models\my_hand_gesture_model\ (not eval, should be something like 1642800231) and paste in
scripts\handGestureModule.py in the first variable named "lastModel"

## Dataset
This specific dataset seperates hand gestures into 4 different classes.

- stop
- onefingerup
- twofingerup
- thumbsup

The information about each hand gesture is contained into 42 features.

In [3]:
CSV_COLUMN_NAMES = np.arange(43)
CSV_COLUMN_NAMES = [str(item) for item in CSV_COLUMN_NAMES]
CSV_COLUMN_NAMES[42] = "Species"
SPECIES = ['stop', 'onefingerup', 'twofingerup', 'thumbsup']
CSV_PATH = os.path.join("src", "dataHandGesture", "file_0.csv")

In [4]:
df = pd.read_csv(CSV_PATH, names=CSV_COLUMN_NAMES, header=0)
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)

In [5]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,Species
40,0.022010,-0.999758,0.250108,-0.587854,0.259214,-0.191691,0.019232,0.084557,-0.233867,0.120053,...,-0.152029,-0.373072,-0.283672,-0.430019,-0.064061,-0.342360,-0.138294,-0.248837,-0.244318,2.0
39,0.006925,-0.999976,0.238772,-0.643926,0.288878,-0.247537,0.047053,0.039935,-0.198594,0.121888,...,-0.091062,-0.366534,-0.275775,-0.396045,-0.061551,-0.325770,-0.133313,-0.236810,-0.256242,2.0
63,0.082444,-0.905387,-0.195562,-0.609048,-0.501244,-0.195654,-0.739761,0.075124,-0.976515,0.215448,...,-0.054624,0.217127,-0.251185,0.551369,0.118667,0.527922,-0.077365,0.412454,-0.222450,3.0
43,-0.042440,-0.999099,0.244106,-0.771909,0.369299,-0.439311,0.351504,-0.155267,0.200043,0.035764,...,-0.191995,-0.397723,-0.256695,-0.306017,0.036389,-0.157286,-0.053099,-0.062908,-0.237535,2.0
53,-0.139310,-0.474512,0.136706,-0.528648,0.339713,-0.510142,0.248936,-0.525794,0.026963,-0.498081,...,-0.528187,-0.518349,0.001749,-0.452798,-0.171152,-0.326206,-0.427006,-0.191328,-0.571459,2.0


In [6]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train_y = train_y.apply(np.int32)
test_y = test_y.apply(np.int32)
train.head() # the species column is now gone

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
40,0.022010,-0.999758,0.250108,-0.587854,0.259214,-0.191691,0.019232,0.084557,-0.233867,0.120053,...,-0.060401,-0.152029,-0.373072,-0.283672,-0.430019,-0.064061,-0.342360,-0.138294,-0.248837,-0.244318
39,0.006925,-0.999976,0.238772,-0.643926,0.288878,-0.247537,0.047053,0.039935,-0.198594,0.121888,...,-0.051248,-0.091062,-0.366534,-0.275775,-0.396045,-0.061551,-0.325770,-0.133313,-0.236810,-0.256242
63,0.082444,-0.905387,-0.195562,-0.609048,-0.501244,-0.195654,-0.739761,0.075124,-0.976515,0.215448,...,0.279709,-0.054624,0.217127,-0.251185,0.551369,0.118667,0.527922,-0.077365,0.412454,-0.222450
43,-0.042440,-0.999099,0.244106,-0.771909,0.369299,-0.439311,0.351504,-0.155267,0.200043,0.035764,...,0.043580,-0.191995,-0.397723,-0.256695,-0.306017,0.036389,-0.157286,-0.053099,-0.062908,-0.237535
53,-0.139310,-0.474512,0.136706,-0.528648,0.339713,-0.510142,0.248936,-0.525794,0.026963,-0.498081,...,-0.020084,-0.528187,-0.518349,0.001749,-0.452798,-0.171152,-0.326206,-0.427006,-0.191328,-0.571459


In [7]:
train.shape  # we have 120 entires with 4 features

(63, 42)

## Input Function
Remember that nasty input function we created earlier. Well we need to make another one here! Fortunatly for us this one is a little easier to digest.

In [8]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [9]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='8', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='9', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='10', shape=(1,), default_value=None, dty

# 2. Building the Model
And now we are ready to choose a model. For classification tasks there are variety of different estimators/models that we can pick from. Some options are listed below.

- DNNClassifier (Deep Neural Network)
- LinearClassifier

We can choose either model but the DNN seems to be the best choice. This is because we may not be able to find a linear coorespondence in our data.

In [10]:
MODEL_PATH = os.path.join("Tensorflow", "workspace", "models", "my_hand_gesture_model")

In [11]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between len(SPECIES) classes.
    n_classes=len(SPECIES),
    activation_fn=lambda x: tf.nn.leaky_relu(x, alpha=0.01),
    optimizer=lambda: tf.keras.optimizers.Adam(
           learning_rate=tf.compat.v1.train.exponential_decay(
               learning_rate=0.1,
               global_step=tf.compat.v1.train.get_global_step(),
               decay_steps=10000,
               decay_rate=0.96)
       ),
    model_dir= MODEL_PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'Tensorflow\\workspace\\models\\my_hand_gesture_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training

In [12]:
#handGesture = 5000
orientationSteps = 15000

classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=orientationSteps)
# We include a lambda to avoid creating an inner function previously

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.3418968, step = 0
INFO:tensorflow:global_step/sec: 86.8699
INFO:tensorflow:loss = 1.2553983e-06, step = 100 (1.153 sec)
INFO:tensorflow:global_step/sec: 116.4
INFO:tensorflow:loss = 3.7066343e-07, step = 200 (0.860 sec)
INFO:tensorflow:global_step/sec: 136.969
INFO:tensorflow:loss = 3.1711372e-07, step = 300 (0.729 sec)
INFO:ten

## Export Model

In [13]:
# To export
feature_spec = tf.feature_column.make_parse_example_spec(my_feature_columns);
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec);
export_path = classifier.export_saved_model(MODEL_PATH, export_input_fn, as_text=True);

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-15000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: Tensorflow\workspace\models\my_hand_gesture_model\temp-1642800231\saved_model.pbtxt


## Load Model

In [14]:
# Loading the estimator
predict_fn = tf.saved_model.load(export_path).signatures['predict']
predict_fn

<ConcreteFunction pruned(examples) at 0x3A6CDB50>

In [15]:
# Convert input data into serialized Example strings.
examples = []
for index, row in test.iterrows():
    feature = {}
    for col, value in row.iteritems():
        feature[col] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    example = tf.train.Example(
        features=tf.train.Features(
            feature=feature
        )
    )
    examples.append(example.SerializeToString())

# Convert from list to tensor
examples = tf.constant(examples)

print(test)
print(feature)

           0         1         2         3         4         5         6  \
1   0.013499 -0.999909  0.464612 -0.739300  0.716077 -0.329336  0.787706   
6  -0.001501 -0.999999  0.250878 -0.788306  0.432767 -0.513341  0.508903   
7  -0.009106 -0.999958  0.294411 -0.896518  0.512861 -0.612737  0.625829   
9  -0.035534 -0.999368  0.287749 -0.892991  0.520524 -0.656249  0.645677   
11  0.021312 -0.999773  0.267268 -0.724411  0.361117 -0.367756  0.432624   
14 -0.010063 -0.579032  0.325405 -0.514793  0.598796 -0.346519  0.794756   
16 -0.090376 -0.876720  0.158255 -0.958652  0.401871 -0.915697  0.416480   
26  0.155463 -0.770752  0.369157 -0.616812  0.481552 -0.298833  0.432183   
27  0.114149 -0.957719  0.340919 -0.635271  0.457307 -0.295634  0.410503   
42  0.008437 -0.999964  0.300418 -0.690328  0.390240 -0.307137  0.217534   
51 -0.122558 -0.743640  0.113327 -0.577301  0.309023 -0.355065  0.380628   
55 -0.043268 -0.927956  0.161223 -0.851633  0.209151 -0.641365 -0.029022   
56 -0.069918

In [16]:
# make predictions of all testset
predictions = predict_fn(examples=examples)
print(predictions)

{'classes': <tf.Tensor: shape=(16, 1), dtype=string, numpy=
array([[b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'1'],
       [b'1'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'3'],
       [b'3']], dtype=object)>, 'all_class_ids': <tf.Tensor: shape=(16, 4), dtype=int32, numpy=
array([[0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3]])>, 'logits': <tf.Tensor: shape=(16, 4), dtype=float32, numpy=
array([[ 13.307984 , -24.713167 , -46.228035 , -30.422209 ],
       [ 14.833571 , -24.578506 , -30.266556 , -26.462145 ],
       [ 19.465178 , -31.067694 , -33.953358 , -33.324333 ],
       [ 19.618795 , -31.592398 , -33.

In [17]:
# print results
for idx, resultPred in enumerate(predictions["class_ids"]):
    class_id = resultPred[0]
    probability = predictions['probabilities'][idx][class_id]
    print(probability)
    print(f"\tPrediction is {SPECIES[class_id]} {100 * probability :.2f}%")
    print(f"\tExpected: {SPECIES[test_y.iloc[idx]]}")

tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is stop 100.00%
	Expected: stop
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is stop 100.00%
	Expected: stop
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is stop 100.00%
	Expected: stop
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is stop 100.00%
	Expected: stop
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is stop 100.00%
	Expected: stop
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is stop 100.00%
	Expected: stop
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is stop 100.00%
	Expected: stop
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is onefingerup 100.00%
	Expected: onefingerup
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is onefingerup 100.00%
	Expected: onefingerup
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is twofingerup 100.00%
	Expected: twofingerup
tf.Tensor(1.0, shape=(), dtype=float32)
	Prediction is twofingerup 100.00%
	Expected: twofingerup
tf.Tensor(1.0, shape

## Evaluation

In [18]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-01-21T22:23:56
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.49806s
INFO:tensorflow:Finished evaluation at 2022-01-21-22:23:56
INFO:tensorflow:Saving dict for global step 15000: accuracy = 1.0, average_loss = 0.0, global_step = 15000, loss = 0.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-15000

Test set accuracy: 1.000



## Predictions
Now that we have a trained model it's time to use it to make predictions. I've written a little script below that allows you to type the features of a flower and see a prediction for its class.

In [19]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = CSV_COLUMN_NAMES[:-1]
predict = {}

for rowNum in range(5):
    expected = SPECIES[test_y.iloc[rowNum]]
    for idx, feature in enumerate(features):
      predict[feature] = [test.iloc[rowNum][idx]] #to predict the first row of test

    predictions = classifier.predict(input_fn=lambda: input_fn(predict))
    for pred_dict in predictions:
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print("\n")
        print('Prediction is "{}" ({:.1f}%)'.format(
            SPECIES[class_id], 100 * probability))
        print(f"Expected is {expected}")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "stop" (100.0%)
Expected is stop
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "stop" (100.0%)
Expected is stop
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_